# LangGraph Tutorial: Building Agentic Workflows with Tool Calling

## What You'll Build
A financial assistant agent that uses LangGraph to intelligently execute calculations:
- Currency conversions (multi-currency support)
- EMI (loan payment) calculations
- Compound interest computations
- Sequential and parallel tool execution

## What is LangGraph?
LangGraph enables building **stateful, multi-actor applications** with LLMs using **graph-based workflows**. 

### Why LangGraph vs. Simple LangChain Chains?

**Traditional LangChain Chains:**
- Linear execution: A → B → C
- No cycles or loops
- Limited conditional logic
- State doesn't persist naturally

**LangGraph Advantages:**
- ✅ **Cycles**: Loop back for iterative reasoning
- ✅ **Conditional Routing**: Dynamic path selection based on LLM decisions
- ✅ **State Management**: Persistent conversation context
- ✅ **Parallel Execution**: Run independent tools simultaneously
- ✅ **Human-in-the-Loop**: Pause for approval (advanced feature)

### Real-World Use Cases
- Customer support chatbots with tool access (search, ticketing, etc.)
- Research agents that browse, summarize, and synthesize information
- Financial advisors that fetch data, calculate, and recommend
- Code assistants that analyze, execute, and debug

## Learning Objectives

By completing this tutorial, you will:

1. **Tool Creation**
   - Create custom LangChain tools with proper type hints
   - Implement input validation and error handling
   - Write comprehensive docstrings for tool discovery

2. **Graph Architecture**
   - Build a StateGraph with agent and tool nodes
   - Configure conditional routing based on LLM decisions
   - Understand state management in graph workflows

3. **Execution Patterns**
   - Execute single tool calls
   - Chain multiple sequential tool calls (dependent tasks)
   - Trigger parallel tool execution (independent tasks)
   - Maintain conversational context across turns

4. **Observability**
   - Track execution flow through graph nodes
   - Monitor tool calls and responses
   - Understand the agent's decision-making process

## Prerequisites
- Basic Python knowledge
- Familiarity with LangChain concepts (LLMs, prompts, chains)
- Understanding of async/await (helpful but not required)

## Estimated Time: 90 minutes

## LangGraph Architecture Overview

### The Agentic Loop

In [ ]:
def render_mermaid(diagram_code, width=400):
    '''Helper function to render Mermaid diagrams using mermaid.ink'''
    from IPython.display import Image, display
    import base64
    import urllib.parse
    
    # Encode diagram for URL
    graphbytes = diagram_code.encode('utf-8')
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode('ascii')
    
    # Use mermaid.ink service
    url = f'https://mermaid.ink/img/{base64_string}'
    
    # Display as image
    display(Image(url=url, width=width))

In [ ]:
# Render Mermaid diagram
render_mermaid('''graph TD
    START([START]) --> AGENT[Agent Node: call_llm]
    AGENT --> ROUTER{Router: should_continue?}
    ROUTER -->|Tool calls present| TOOLS[Tools Node: Execute]
    ROUTER -->|No tool calls| END([END])
    TOOLS --> AGENT
    
    style START fill:#90EE90
    style END fill:#FFB6C1
    style AGENT fill:#87CEEB
    style TOOLS fill:#FFD700
    style ROUTER fill:#DDA0DD''')

### Key Components

1. **State**: Shared data structure (list of messages) that flows through nodes
2. **Nodes**: Functions that read state, process it, and return updates
3. **Edges**: Connections defining flow (conditional or static)
4. **Router**: Conditional edge that determines next node based on state

### What Makes This "Agentic"?

The LLM **autonomously decides**:
- Which tools to call (if any)
- When to call them (sequential vs parallel)
- When it has enough information to respond
- How many reasoning loops it needs

You define the tools and graph structure; the LLM orchestrates execution.

---
# SECTION 2: Environment Setup

## Required Environment Variables

Create a `.env` file with:

```env
# GCP Vertex AI Configuration
GOOGLE_PROJECT_ID=your-project-id
GOOGLE_REGION=us-central1
GOOGLE_APPLICATION_CREDENTIALS=/path/to/service-account.json

# LangSmith Observability (Optional but Recommended)
LANGSMITH_API_KEY=your-langsmith-key
LANGSMITH_PROJECT=langgraph-tutorial
LANGCHAIN_TRACING_V2=true
```

### Why These Variables?

- **GOOGLE_PROJECT_ID**: GCP project containing Vertex AI resources
- **GOOGLE_REGION**: Data residency and latency optimization
- **GOOGLE_APPLICATION_CREDENTIALS**: Service account for authentication
- **LANGSMITH_***: Enables execution tracing and debugging dashboards

### Installation

```bash
pip install langchain langchain-google-vertexai langgraph python-dotenv
```

In [ ]:
# Core LangChain imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# LangGraph imports
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

# LLM Provider
from langchain_google_genai import ChatGoogleGenerativeAI

# Environment management
import os
from dotenv import load_dotenv

# Utilities
from typing import Literal

# For displaying Mermaid diagrams
from IPython.display import HTML, display

In [ ]:
# Load environment variables
load_dotenv("../../.env")  # Adjust path to your .env location

# Verify critical variables are loaded
required_vars = ["GOOGLE_PROJECT_ID", "GOOGLE_REGION"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    raise EnvironmentError(f"Missing required environment variables: {', '.join(missing_vars)}")

print("✅ Environment loaded successfully")
print(f"   Project: {os.getenv('GOOGLE_PROJECT_ID')}")
print(f"   Region: {os.getenv('GOOGLE_REGION')}")
print(f"   LangSmith Tracing: {'Enabled' if os.getenv('LANGCHAIN_TRACING_V2') else 'Disabled'}")

---
# SECTION 3: Tool Creation Fundamentals

## Why Create Custom Tools?

### LLM Limitations

LLMs **cannot** directly perform:
- ❌ Real-time calculations (arithmetic, financial formulas)
- ❌ External API calls (weather, stock prices, database queries)
- ❌ File system operations (read/write files)
- ❌ Time-sensitive operations (current date, scheduling)

### The Solution: Tools

In [ ]:
# Render Mermaid diagram
render_mermaid('''graph LR
    A[LLM Request] --> B{Need Tools?}
    B -->|Yes| C[Generate Tool Call]
    C --> D[Execute Python Function]
    D --> E[Return Result]
    E --> F[LLM Synthesizes Response]
    B -->|No| F
    
    style A fill:#87CEEB
    style D fill:#FFD700
    style F fill:#90EE90''', 1200)

Tools are Python functions that:
1. Extend LLM capabilities with executable code
2. Have clear schemas (type hints) so LLMs know when to use them
3. Return structured results the LLM can incorporate into responses

### Tool Decorator Magic

The `@tool` decorator:
- Auto-generates JSON schema from docstring
- Enables LLM to discover tool capabilities
- Handles serialization/deserialization
- Provides validation and error handling hooks

### Best Practices

✅ **DO:**
- Write comprehensive docstrings (LLM reads these!)
- Use type hints for all parameters
- Validate inputs and return clear error messages
- Return human-readable strings, not raw data structures

❌ **DON'T:**
- Use vague parameter names ("x", "val", "data")
- Skip error handling
- Return complex objects (LLMs prefer text)
- Make assumptions about input formats

In [ ]:
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Args:
        amount: The amount to convert
        from_currency: Source currency code (USD, EUR, GBP, INR, JPY)
        to_currency: Target currency code (USD, EUR, GBP, INR, JPY)
    
    Returns:
        A string with the conversion result including the exchange rate
    """
        
    # Simplified exchange rates (relative to USD)
    # In production, you would fetch real-time rates from an API
    exchange_rates = {
        "USD": 1.0,
        "EUR": 0.92,
        "GBP": 0.79,
        "INR": 83.12,
        "JPY": 149.50
    }
    
    # Validate currencies
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates:
        return f"Error: Unsupported currency {from_currency}. Supported: {', '.join(exchange_rates.keys())}"
    
    if to_currency not in exchange_rates:
        return f"Error: Unsupported currency {to_currency}. Supported: {', '.join(exchange_rates.keys())}"
    
    # Convert to USD first, then to target currency
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    
    # Calculate the effective exchange rate
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    result = (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )
    
    return result

In [ ]:
type(currency_converter)

In [ ]:
# Test the tool directly (before integrating with graph)
print("=" * 70)
print("TESTING: Currency Converter")
print("=" * 70)

# Test Case 1: Standard conversion
print("\n[Test 1] Convert 1000 USD to EUR")
result = currency_converter.invoke({"amount": 1000, "from_currency": "USD", "to_currency": "EUR"})
print(result)

# Test Case 2: Error handling - invalid currency
print("\n[Test 2] Invalid currency (XYZ)")
result = currency_converter.invoke({"amount": 100, "from_currency": "XYZ", "to_currency": "USD"})
print(result)

# Test Case 3: Same currency conversion
print("\n[Test 3] Convert INR to INR (edge case)")
result = currency_converter.invoke({"amount": 5000, "from_currency": "INR", "to_currency": "INR"})
print(result)

In [ ]:
@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Args:
        principal: The loan amount (principal)
        annual_interest_rate: Annual interest rate as a percentage (e.g., 8.5 for 8.5%)
        tenure_months: Loan tenure in months
    
    Returns:
        A string with the EMI calculation details
    """

    # Validate inputs
    if principal <= 0:
        return "Error: Principal must be greater than 0"
    if annual_interest_rate < 0:
        return "Error: Interest rate cannot be negative"
    if tenure_months <= 0:
        return "Error: Tenure must be greater than 0"
    
    # Calculate monthly interest rate
    monthly_rate = annual_interest_rate / 12 / 100
    
    # Calculate EMI using the formula
    if monthly_rate == 0:
        # If interest rate is 0, EMI is simply principal divided by tenure
        emi = principal / tenure_months
    else:
        # Standard EMI formula: EMI = P × r × (1 + r)^n / ((1 + r)^n - 1)
        emi = principal * monthly_rate * (1 + monthly_rate) ** tenure_months / \
              ((1 + monthly_rate) ** tenure_months - 1)
    
    # Calculate total payment and total interest
    total_payment = emi * tenure_months
    total_interest = total_payment - principal
    
    # Format the result
    result = (
        f"EMI Calculation Result:\n"
        f"  Principal: {currency} {principal:,.2f}\n"
        f"  Annual Interest Rate: {annual_interest_rate}%\n"
        f"  Tenure: {tenure_months} months ({tenure_months//12} years {tenure_months%12} months)\n"
        f"  ---\n"
        f"  Monthly EMI: {currency} {emi:,.2f}\n"
        f"  Total Payment: {currency} {total_payment:,.2f}\n"
        f"  Total Interest: {currency} {total_interest:,.2f}"
    )
    
    return result

In [ ]:
type(emi_calculator)

In [ ]:
# Test the EMI calculator
print("=" * 70)
print("TESTING: EMI Calculator")
print("=" * 70)

# Test Case 1: Standard home loan scenario
print("\n[Test 1] Home Loan: ₹50L at 8.5% for 5 years")
result = emi_calculator.invoke({"principal": 500000, "annual_interest_rate": 8.5, "tenure_months": 60, "currency": "INR"})
print(result)

# Test Case 2: Car loan scenario
print("\n[Test 2] Car Loan: ₹8L at 9% for 3 years")
result = emi_calculator.invoke({"principal": 800000, "annual_interest_rate": 9, "tenure_months": 36, "currency": "INR"})
print(result)

# Test Case 3: Zero interest (edge case)
print("\n[Test 3] Zero Interest Loan")
result = emi_calculator.invoke({"principal": 100000, "annual_interest_rate": 0, "tenure_months": 12, "currency": "INR"})
print(result)

## ✅ Checkpoint 1: Verify Tools Work

Before proceeding, ensure:
- [ ] Currency converter executes without errors
- [ ] EMI calculator produces correct output  
- [ ] You understand how `@tool` decorator works
- [ ] Error handling catches invalid inputs

**What You've Learned:**
- Tools extend LLM capabilities with Python code
- Type hints + docstrings = automatic schema generation
- Return human-readable strings for LLM integration
- Input validation prevents runtime errors

**Next Step:** Build the LangGraph that lets the LLM decide when to use these tools!

---
# SECTION 4: Building the LangGraph

## State Management in LangGraph

### What is State?

State is the **shared data structure** that flows through all nodes in the graph. It acts as:
- Memory for the conversation
- Container for passing data between nodes
- History of all interactions

In [ ]:
# Render Mermaid diagram
render_mermaid('''graph LR
    A[Initial State] --> B[Node 1: Agent]
    B --> C[Updated State]
    C --> D[Node 2: Tools]
    D --> E[Updated State]
    E --> F[Node 3: Agent]
    F --> G[Final State]
    
    style A fill:#90EE90
    style C fill:#FFD700
    style E fill:#FFD700
    style G fill:#FFB6C1''', 1200)

### MessagesState

LangGraph provides `MessagesState` which contains:
- `messages`: List of all conversation messages

### Message Types

1. **HumanMessage**: User input
   ```python
   HumanMessage(content="Convert 100 USD to EUR")
   ```

2. **AIMessage**: LLM response (can include tool_calls)
   ```python
   AIMessage(content="Let me convert that for you", tool_calls=[...])
   ```

3. **ToolMessage**: Tool execution result
   ```python
   ToolMessage(content="100 USD = 92 EUR", tool_call_id="...")
   ```

### How State Updates

Each node:
1. **Reads** current state (`state["messages"]`)
2. **Processes** the data (call LLM, execute tools, etc.)
3. **Returns** updates (new messages to append)

LangGraph **automatically merges** the updates into state before passing to the next node.

In [ ]:
type(currency_converter)

In [ ]:
type(emi_calculator)

In [ ]:
# Initialize the LLM with tool binding
print("Initializing Vertex AI LLM...")

# Create the base LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3,  # Lower temperature for more consistent, factual responses
    max_tokens=1024,
    project=os.getenv("GOOGLE_PROJECT_ID"),
    location=os.getenv("GOOGLE_REGION")
)

# Define the tools list
tools = [currency_converter, emi_calculator]

# Bind tools to the LLM
# This tells the LLM: "You have access to these tools, and here are their schemas"
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools:")
for tool in tools:
    print(f"   • {tool.name}: {tool.description[:60]}...")

In [ ]:
# Define the agent node that calls the LLM
def call_llm(state: MessagesState):
    """
    Agent node that invokes the LLM with the current conversation state.
    
    The LLM will analyze the conversation and decide to either:
    1. Call one or more tools (returns AIMessage with tool_calls)
    2. Provide a final response (returns AIMessage with content only)
    
    Args:
        state: Current graph state containing message history
        
    Returns:
        Dictionary with "messages" key containing the LLM response
    """
    # Invoke the LLM with the full conversation history
    response = llm_with_tools.invoke(state["messages"])
    
    # Return the response wrapped in the expected format
    # LangGraph will automatically append this to state["messages"]
    return {"messages": [response]}

print("✅ Agent node (call_llm) defined")

In [ ]:
# Define the conditional edge logic
def should_continue(state: MessagesState) -> Literal["tools", END]:
    """
    Router that determines the next node based on LLM's decision.
    
    Checks the last message in the conversation:
    - If it contains tool_calls → route to "tools" node
    - Otherwise → route to END (finish execution)
    
    Args:
        state: Current graph state
        
    Returns:
        Either "tools" or END constant
    """
    last_message = state["messages"][-1]
    
    # If the LLM made tool calls, execute them next
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    
    # Otherwise, we're done - return final response
    return END

print("✅ Router (should_continue) defined")

In [ ]:
# Create the graph
print("\nBuilding the StateGraph...")

# Initialize the graph with MessagesState
workflow = StateGraph(MessagesState)

# Add nodes
workflow.add_node("agent", call_llm)  # The LLM agent
workflow.add_node("tools", ToolNode(tools))  # Tool execution node

# Add edges
workflow.add_edge(START, "agent")  # Always start with the agent

# Add conditional edge from agent
workflow.add_conditional_edges(
    "agent",  # Use this function to decide
    should_continue,  # Use this function to decide
    # Possible destinations:
    {
        "tools": "tools",  # If should_continue returns "tools", go to tools node
        END: END  # If should_continue returns END, finish
    }
)

# After tools execute, always loop back to agent
workflow.add_edge("tools", "agent")

# Compile the graph into an executable app
app = workflow.compile()

print("✅ Graph compiled successfully!")
print("\nGraph Structure:")
print("  START → agent → [conditional router]")
print("            ↓              ↓")
print("          tools ←──────── END")
print("            ↓")
print("          agent (loop back)")

In [ ]:
mermaid_diagram = app.get_graph().draw_mermaid()
render_mermaid(mermaid_diagram)

## Understanding the Graph Flow

### What Just Happened?

We built a graph with:

1. **2 Nodes:**
   - `agent`: Calls the LLM to decide what to do
   - `tools`: Executes the tools the LLM requested

2. **3 Edge Types:**
   - **Static edge** (START → agent): Always go here first
   - **Conditional edge** (agent → tools/END): Router decides based on LLM response
   - **Static edge** (tools → agent): Always loop back after tool execution

3. **The Cycle:**
   ```
   agent → tools → agent → tools → agent → END
   ```
   This allows multi-step reasoning:
   - LLM calls tool
   - Tool returns result
   - LLM sees result, decides if it needs more tools
   - Repeats until LLM has enough info to respond

### Why This Works

In [ ]:
# Render Mermaid diagram
render_mermaid('''sequenceDiagram
    participant U as User
    participant A as Agent (LLM)
    participant T as Tools
    
    U->>A: Convert 100 USD to EUR
    A->>A: Analyze query
    A->>T: Call currency_converter
    T->>T: Execute conversion
    T->>A: Return 100 USD = 92 EUR
    A->>A: Synthesize response
    A->>U: 100 USD equals 92 EUR''', 900)

The LLM acts as the **orchestrator**:
- It sees the full conversation (including tool results)
- It autonomously decides when to use tools
- It determines when it has enough information
- It controls the execution flow through its responses

**You** define the tools and graph structure.  
**The LLM** decides the execution path.

---
# SECTION 5: Execution Examples

## Testing the Agentic Graph

We'll test 5 different execution patterns:

1. **Single Tool Call**: Simple, one-step task
2. **Multiple Sequential Calls**: Agent loops to gather more info
3. **Dependent Sequential Tasks**: Second task needs first result
4. **Conversational Context**: Multi-turn conversation with state
5. **Parallel Tool Execution**: Independent tasks run simultaneously

Each example demonstrates different aspects of the agent's decision-making.

In [ ]:
print("=" * 70)
print("EXAMPLE 1: Single Tool Call")
print("=" * 70)
print("\nQuery: 'What is 1000 USD in EUR?'\n")

In [ ]:
# Create initial state with user message
state = {"messages": [HumanMessage(content="What is 1000 USD in EUR?")]}

In [ ]:
# Execute the graph
result = app.invoke(state)

In [ ]:
result

In [ ]:
# Display the conversation flow
print("CONVERSATION FLOW:")
print("-" * 70)
for i, msg in enumerate(result["messages"], 1):
    if isinstance(msg, HumanMessage):
        print(f"\n[{i}] 👤 USER:")
        print(f"    {msg.content}")
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"\n[{i}] 🤖 AGENT (calling tools):")
            for tc in msg.tool_calls:
                print(f"    → Tool: {tc['name']}")
                print(f"      Args: {tc['args']}")
        else:
            print(f"\n[{i}] 🤖 AGENT (final response):")
            print(f"    {msg.content}")
    elif isinstance(msg, ToolMessage):
        print(f"\n[{i}] 🔧 TOOL RESULT:")
        print(f"    {msg.content}")

print("\n" + "=" * 70)
print("ANALYSIS:")
print("=" * 70)
print("✅ Single tool call pattern")
print("✅ Agent identified need for currency_converter")
print("✅ Tool executed and returned result")
print("✅ Agent formulated natural language response")

In [ ]:
print("=" * 70)
print("EXAMPLE 2: Multiple Sequential Tool Calls (Information Gathering)")
print("=" * 70)
print("\nQuery: 'Convert 1000 USD to EUR and also calculate EMI for USD 500000 at 8.5% for 5 years'\n")

In [ ]:
# Create initial state
state = {
    "messages": [
        HumanMessage(content="Convert 1000 USD to EUR and also calculate EMI for USD 500000 at 8.5% for 5 years")
    ]
}

In [ ]:
app.invoke(state)

In [ ]:
app.stream(state)

In [ ]:

# Execute with streaming to see each step
print("EXECUTION STEPS:")
print("-" * 70)

step_count = 0
for event in app.stream(state):
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: {node_name}")
        
        if "messages" in data:
            last_msg = data["messages"][-1]
            
            if isinstance(last_msg, AIMessage) and hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
                print(f"  🤖 Agent Decision: Call {len(last_msg.tool_calls)} tool(s)")
                for tc in last_msg.tool_calls:
                    print(f"     • {tc['name']}({tc['args']})")
                    
            elif isinstance(last_msg, ToolMessage):
                print(f"  🔧 Tool Execution Complete")
                print(f"     Result preview: {last_msg.content}...")
                
            elif isinstance(last_msg, AIMessage):
                print(f"  💬 Final Response Generated")

# Get final result
result = app.invoke(state)

print("\n" + "=" * 70)
print("FINAL RESPONSE:")
print("=" * 70)
content = result["messages"][-1].content
if type(content) == list:
    print(content[0]['text'])
else:
    print(content)

print("\n" + "=" * 70)
print("ANALYSIS:")
print("=" * 70)
print("✅ Agent handled multi-part query")
print("✅ Both tools called in single decision (potentially parallel)")
print("✅ Results synthesized into coherent response")

In [ ]:
print("=" * 70)
print("EXAMPLE 2a: Multiple Sequential Tool Calls (Information Gathering)")
print("=" * 70)
print("\nQuery: 'Convert 1000 USD to EUR and also calculate EMI for USD 500000 at 8.5% for 5 years'\n")

In [ ]:
# Create initial state
state = {
    "messages": [
        HumanMessage(content="Convert 1000 USD to EUR "),
        HumanMessage(content="Calculate EMI for USD 500000 at 8.5% for 5 years")
    ]
}

In [ ]:
app.invoke(state)

In [ ]:

# Execute with streaming to see each step
print("EXECUTION STEPS:")
print("-" * 70)

step_count = 0
for event in app.stream(state):
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: {node_name}")
        
        if "messages" in data:
            last_msg = data["messages"][-1]
            
            if isinstance(last_msg, AIMessage) and hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
                print(f"  🤖 Agent calling: {last_msg.tool_calls[0]['name']}")
                print(f"     Args: {last_msg.tool_calls[0]['args']}")
                    
            elif isinstance(last_msg, ToolMessage):
                print(f"  🔧 Tool Execution Complete")
                print(f"     Result preview: {last_msg.content}...")
                
            elif isinstance(last_msg, AIMessage):
                print(f"  💬 Final Response Generated")

In [ ]:
# Get final result
result = app.invoke(state)

print("\n" + "=" * 70)
print("FINAL RESPONSE:")
print("=" * 70)
content = result["messages"][-1].content
if type(content) == list:
    print(content[0]['text'])
else:
    print(content)

print("\n" + "=" * 70)
print("ANALYSIS:")
print("=" * 70)
print("✅ Agent handled multi-part query")
print("✅ Both tools called in single decision (potentially parallel)")
print("✅ Results synthesized into coherent response")

In [ ]:
print("=" * 70)
print("EXAMPLE 3: Dependent Sequential Execution")
print("=" * 70)
print("\nQuery: 'Convert 100000 INR to USD, then calculate EMI for THAT amount at 7% for 24 months'\n")
print("NOTE: The second task DEPENDS on the first result - must execute sequentially!\n")

In [ ]:

# Create initial state
state = {
    "messages": [
        HumanMessage(
            content="Convert 100000 INR to USD, then calculate EMI for that USD amount at 7% interest for 24 months"
        )
    ]
}

In [ ]:
app.invoke(state)

In [ ]:

# Execute with streaming
print("EXECUTION TRACE:")
print("-" * 70)

step_count = 0
for event in app.stream(state):
    print(event.items())
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: {node_name}")
        
        if "messages" in data:
            last_msg = data["messages"][-1]
            
            if isinstance(last_msg, AIMessage) and hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
                print(f"  🤖 Agent calling: {last_msg.tool_calls[0]['name']}")
                print(f"     Args: {last_msg.tool_calls[0]['args']}")
                
            elif isinstance(last_msg, ToolMessage):
                print(f"  ✅ Tool completed")
                print(f"     Output: {last_msg.content[:150]}...")
                
            elif isinstance(last_msg, AIMessage):
                print(f"  💬 Agent: Task complete!")


In [ ]:
# Get final result
result = app.invoke(state)

In [ ]:

print("\n" + "=" * 70)
print("FINAL RESPONSE:")
print("=" * 70)
final_message = result["messages"][-1]
print(final_message.content)

print("\n" + "=" * 70)
print("ANALYSIS:")
print("=" * 70)
print("✅ Agent recognized task dependency")
print("✅ First: Convert INR → USD")
print("✅ Then: Use converted amount in EMI calculation")
print("✅ Sequential execution required (not parallel)")
print("\n💡 Key Insight: The agent automatically detected that the second")
print("   task depends on the first result and executed them in sequence!")

In [ ]:
print("=" * 70)
print("EXAMPLE 4: Conversational Context (Multi-Turn)")
print("=" * 70)
print("\nThis demonstrates how state persists across multiple interactions.\n")

In [ ]:

# Turn 1: Initial query
print("[TURN 1]")
print("-" * 70)
state = {"messages": [HumanMessage(content="Convert 5000 GBP to INR")]}
result = app.invoke(state)
print(f"👤 User: {state['messages'][0].content}")
print(f"🤖 Assistant: {result['messages'][-1].content}")

In [ ]:
result

In [ ]:

# Turn 2: Follow-up query that references previous result
print("\n[TURN 2]")
print("-" * 70)
print("👤 User: Now calculate EMI for that INR amount at 9% for 2 years")
result["messages"].append(
    HumanMessage(content="Now calculate EMI for that INR amount at 9% for 2 years")
)

In [ ]:
result = app.invoke(result)
print(f"🤖 Assistant: {result['messages'][-1].content}")

In [ ]:
result

In [ ]:

print("\n" + "=" * 70)
print("ANALYSIS:")
print("=" * 70)
print("✅ Agent maintained context from Turn 1")
print("✅ Understood 'that INR amount' referred to previous conversion result")
print("✅ State contains full conversation history")
print("\n💡 Key Insight: MessagesState automatically preserves all messages,")
print("   enabling natural conversational flow with context awareness!")

In [ ]:
print("=" * 70)
print("EXAMPLE 5: Parallel Tool Execution")
print("=" * 70)
print("\nQuery: 'Convert 500000 INR to USD AND ALSO compute EMI for 500000 at 8.5% for 24 months'\n")
print("NOTE: Both tasks are INDEPENDENT - can execute in parallel!\n")

In [ ]:
# Create initial state
input_message = HumanMessage(
    content="Convert INR 500,000 to USD and also compute EMI at 8.5% for a 24 month loan of INR 200,000"
)
initial_state = {"messages": [input_message]}

print(f"User Query: {input_message.content}\n")

In [ ]:
app.invoke(initial_state)

In [ ]:

# Execute with streaming to observe parallel execution
print("PROCESSING STEPS:")
print("-" * 70)

step_count = 0
for event in app.stream(initial_state):
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: {node_name}")
        
        if "messages" in data:
            last_msg = data["messages"][-1]
            
            if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
                # Check if multiple tools are called at once
                if len(last_msg.tool_calls) > 1:
                    print(f"  🚀 PARALLEL Execution: {len(last_msg.tool_calls)} tools called simultaneously!")
                    for tc in last_msg.tool_calls:
                        print(f"     • {tc['name']}")
                        print(f"       Args: {tc['args']}")
                else:
                    for tc in last_msg.tool_calls:
                        print(f"  🔧 Tool Call: {tc['name']}")
                        print(f"     Args: {tc['args']}")
                        
            elif hasattr(last_msg, "content") and last_msg.content:
                print(f"  💬 Response: {last_msg.content}")

# Get final result
result = app.invoke(initial_state)

print("\n" + "=" * 70)
print("FINAL RESPONSE:")
print("=" * 70)
final_message = result["messages"][-1]
print(final_message.content)

print("\n" + "=" * 70)
print("KEY INSIGHT: Parallel vs Sequential Execution")
print("=" * 70)
print("\n📊 COMPARISON:\n")
print("Example 3 (Sequential):")
print("  Query: 'Convert INR to USD THEN calculate EMI for THAT amount'")
print("  → Second task depends on first result")
print("  → MUST run sequentially")
print("  → Execution: Tool1 → Agent → Tool2 → Agent → Response")
print("\nExample 5 (Parallel):")
print("  Query: 'Convert INR to USD AND ALSO compute EMI for 500k'")
print("  → Both tasks are independent")
print("  → CAN run in parallel")
print("  → Execution: Tool1 + Tool2 (simultaneous) → Agent → Response")
print("\n💡 The agent intelligently analyzes task dependencies and")
print("   automatically chooses sequential or parallel execution!")

---
# SECTION 6: Understanding Graph Execution

## How the Graph Executes: Deep Dive

### Execution Lifecycle

In [ ]:
# Render Mermaid diagram
render_mermaid('''sequenceDiagram
    participant User
    participant Graph
    participant Agent
    participant Router
    participant Tools
    
    User->>Graph: invoke(state)
    Graph->>Agent: state with user message
    Agent->>Agent: LLM analyzes query
    Agent->>Graph: AIMessage (with/without tool_calls)
    Graph->>Router: Check last message
    
    alt Has tool_calls
        Router->>Tools: Execute tools
        Tools->>Tools: Run Python functions
        Tools->>Graph: ToolMessage(s) with results
        Graph->>Agent: Updated state (loop back)
        Agent->>Agent: LLM sees tool results
        Agent->>Graph: AIMessage (decision)
        Graph->>Router: Check again
    end
    
    Router->>Graph: Route to END
    Graph->>User: Final state returned''', 900)

### Key Observations

**State is Immutable Per Node**
- Nodes return updates, not modified state
- LangGraph merges updates automatically
- Previous messages never disappear

**Cycles Enable Reasoning**
- Tools → Agent loop allows multi-step tasks
- Agent can call tools multiple times
- Terminates when agent is satisfied

**Streaming Shows Real-Time Progress**
- `.stream()` yields each node's output
- Useful for debugging and UX (loading indicators)
- `.invoke()` waits for completion

**Parallel Tool Calls**
- LLM can include multiple tool_calls in one AIMessage
- ToolNode executes them all
- Results appended in order

In [ ]:
print("""
╔═══════════════════════════════════════════════════════════════════╗
║                  LANGGRAPH FEATURES DEMONSTRATED                  ║
╚═══════════════════════════════════════════════════════════════════╝

✅ STATE MANAGEMENT
   • MessagesState persists conversation history
   • State flows through nodes automatically
   • Immutable updates merged by framework

✅ NODES (Modular Functions)
   • Agent Node: LLM decision-making
   • Tools Node: Execute Python functions
   • Custom logic encapsulated

✅ EDGES (Flow Control)
   • Static: START → agent, tools → agent
   • Conditional: agent → tools/END based on router

✅ CONDITIONAL ROUTING
   • Router function examines state
   • Dynamic path selection
   • Enables agentic autonomy

✅ TOOL CALLING
   • LLM decides when to use tools
   • Type-safe parameter passing
   • Error handling at tool level

✅ CYCLES (Iterative Reasoning)
   • tools → agent → tools loop
   • Multi-step problem solving
   • Terminates when agent decides

✅ STREAMING
   • Real-time execution visibility
   • Observe agent's decision process
   • Useful for debugging and UX

✅ PARALLEL EXECUTION
   • LLM calls multiple tools simultaneously
   • When tasks are independent
   • Automatic orchestration

╔═══════════════════════════════════════════════════════════════════╗
║            WHAT MAKES THIS "AGENTIC"?                             ║
╚═══════════════════════════════════════════════════════════════════╝

The LLM AUTONOMOUSLY decides:
  → Which tools to call (if any)
  → When to call them (sequential vs parallel)
  → How many reasoning loops it needs
  → When it has enough information to respond

YOU define:
  → Available tools (capabilities)
  → Graph structure (possible paths)
  → Routing logic (decision points)

The agent operates within these constraints but controls execution flow.
""")

---
# SECTION 7: Hands-On Exercise

## 🎯 Hands-On Exercise: Compound Interest Calculator

### Objective
Extend the financial assistant by adding a compound interest calculator tool.

### Learning Goals
1. Create a custom tool from scratch
2. Implement financial formula correctly
3. Integrate new tool into existing graph
4. Test with complex queries

### Part 1: Implement the Tool

Complete the function below using the compound interest formula:

```
A = P(1 + r/n)^(nt)
```

Where:
- **A** = Final amount
- **P** = Principal (initial investment)
- **r** = Annual interest rate (as decimal, e.g., 0.06 for 6%)
- **n** = Compounding frequency per year (e.g., 12 for monthly)
- **t** = Time in years

### Expected Output Format

```
Compound Interest Calculation:
  Initial Principal: ₹10,000.00
  Annual Interest Rate: 6%
  Investment Period: 5 years
  Compounding: 12 times/year (Monthly)
  ---
  Final Amount: ₹13,488.50
  Interest Earned: ₹3,488.50
  Effective Annual Rate: 6.17%
```

### Test Cases

**Test 1:** Principal=10000, Rate=6%, Years=5, Compounds=12
- Expected Final: ₹13,488.50

**Test 2:** Principal=50000, Rate=8%, Years=10, Compounds=4
- Expected Final: ₹110,278.85

**Test 3:** Principal=100000, Rate=0%, Years=5, Compounds=12
- Expected Final: ₹100,000.00 (edge case)

In [ ]:
@tool
def compound_interest_calculator(
    principal: float, 
    annual_rate: float, 
    years: int, 
    compounds_per_year: int = 12
) -> str:
    """
    Calculate compound interest on an investment.
    
    Formula: A = P(1 + r/n)^(nt)
    
    Args:
        principal: Initial investment amount
        annual_rate: Annual interest rate as percentage (e.g., 6 for 6%)
        years: Investment period in years
        compounds_per_year: How often interest compounds per year (default: 12 for monthly)
    
    Returns:
        A string with detailed calculation results
    """
    # TODO: Implement input validation
    # Hint: Check that principal > 0, annual_rate >= 0, years > 0, compounds_per_year > 0
    
    # TODO: Convert annual_rate from percentage to decimal
    # Hint: r = annual_rate / 100
    
    # TODO: Calculate final amount using the formula
    # Hint: A = principal * (1 + r/compounds_per_year) ** (compounds_per_year * years)
    
    # TODO: Calculate interest earned
    # Hint: interest = final_amount - principal
    
    # TODO: Calculate effective annual rate
    # Hint: effective_rate = ((1 + r/compounds_per_year) ** compounds_per_year - 1) * 100
    
    # TODO: Format and return the result string
    # Use the expected output format shown above
    
    pass

# ============================================================================
# YOUR CODE BELOW: Uncomment and complete the implementation
# ============================================================================

# # Validation
# if principal <= 0:
#     return "Error: Principal must be greater than 0"
# # ... add more validations
#
# # Convert rate
# r = # YOUR CODE HERE
#
# # Calculate final amount
# final_amount = # YOUR CODE HERE
#
# # Calculate interest earned
# interest_earned = # YOUR CODE HERE
#
# # Calculate effective annual rate
# effective_rate = # YOUR CODE HERE
#
# # Format result
# result = (
#     f"Compound Interest Calculation:\n"
#     # ... complete the formatting
# )
#
# return result

In [ ]:
# Test your compound interest calculator before integrating with graph

print("=" * 70)
print("TESTING: Compound Interest Calculator")
print("=" * 70)

# Test Case 1: Standard case
print("\n[Test 1] ₹10,000 at 6% for 5 years (monthly compounding)")
try:
    result = compound_interest_calculator.invoke({
        "principal": 10000,
        "annual_rate": 6,
        "years": 5,
        "compounds_per_year": 12
    })
    print(result)
    print("✅ Expected final amount: ~₹13,488.50")
except Exception as e:
    print(f"❌ Error: {e}")

# Test Case 2: Quarterly compounding
print("\n[Test 2] ₹50,000 at 8% for 10 years (quarterly compounding)")
try:
    result = compound_interest_calculator.invoke({
        "principal": 50000,
        "annual_rate": 8,
        "years": 10,
        "compounds_per_year": 4
    })
    print(result)
    print("✅ Expected final amount: ~₹110,278.85")
except Exception as e:
    print(f"❌ Error: {e}")

# Test Case 3: Zero interest (edge case)
print("\n[Test 3] ₹100,000 at 0% for 5 years")
try:
    result = compound_interest_calculator.invoke({
        "principal": 100000,
        "annual_rate": 0,
        "years": 5,
        "compounds_per_year": 12
    })
    print(result)
    print("✅ Expected final amount: ₹100,000.00 (no growth)")
except Exception as e:
    print(f"❌ Error: {e}")

## Part 2: Integrate Tool into Graph

Once your tool works correctly, integrate it into the agentic graph.

### Steps:

1. **Add to Tools List**
   ```python
   tools = [currency_converter, emi_calculator, compound_interest_calculator]
   ```

2. **Rebind LLM**
   ```python
   llm_with_tools = llm.bind_tools(tools)
   ```

3. **Rebuild Graph**
   ```python
   # Recreate workflow with updated tools
   workflow = StateGraph(MessagesState)
   workflow.add_node("agent", call_llm)
   workflow.add_node("tools", ToolNode(tools))  # Now includes compound_interest_calculator
   
   workflow.add_edge(START, "agent")
   workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
   workflow.add_edge("tools", "agent")
   
   app = workflow.compile()
   ```

4. **Test with Natural Language**
   ```python
   state = {"messages": [HumanMessage(content="Calculate compound interest on ₹10,000 at 6% for 5 years")]}
   result = app.invoke(state)
   print(result["messages"][-1].content)
   ```

In [ ]:
# TODO: Add your compound_interest_calculator to the tools list
# tools = [currency_converter, emi_calculator]  # Add your tool here!

# TODO: Rebind the LLM with the updated tools list
# llm_with_tools = llm.bind_tools(tools)

# TODO: Rebuild the graph (copy the graph building code from earlier)
# workflow = StateGraph(MessagesState)
# ... add nodes and edges
# app = workflow.compile()

# TODO: Test with a simple query
# state = {"messages": [HumanMessage(content="Calculate compound interest on ₹10,000 at 6% for 5 years")]}
# result = app.invoke(state)
# print(result["messages"][-1].content)

print("⚠️  Complete the TODOs above to integrate your tool!")

## Part 3: Complex Query Challenge

### Challenge Queries

Test your integrated system with these complex scenarios:

**Challenge 1: Sequential Dependency**
```
"If I invest ₹50,000 at 7% compound interest for 3 years with quarterly 
compounding, then take a loan for the final amount at 9% interest for 
2 years, what would be my monthly EMI?"
```

In [ ]:
# Render Mermaid diagram
render_mermaid('''graph LR
    A[Query] --> B[Compound Interest]
    B --> C[Get Final Amount]
    C --> D[EMI Calculator]
    D --> E[Final Response]
    
    style A fill:#87CEEB
    style B fill:#FFD700
    style C fill:#90EE90
    style D fill:#FFD700
    style E fill:#FFB6C1''')

**Expected Flow:**
1. Calculate compound interest → Get final amount
2. Use that amount as loan principal → Calculate EMI

**Challenge 2: Currency + Compound Interest**
```
"Convert 10,000 USD to INR, then calculate compound interest on that 
INR amount at 8% for 5 years with monthly compounding"
```

**Challenge 3: Parallel Execution**
```
"Calculate compound interest on ₹100,000 at 6% for 10 years AND ALSO 
compute EMI for ₹500,000 at 8.5% for 5 years"
```

**Question:** Will this execute sequentially or in parallel? Why?

### Success Criteria

✅ Agent correctly identifies task dependencies  
✅ Sequential tasks execute in proper order  
✅ Parallel tasks execute simultaneously when independent  
✅ Final response synthesizes all results coherently  
✅ No hallucinations - all numbers come from tool results

In [ ]:
# Test your integrated system with the challenges

print("=" * 70)
print("CHALLENGE 1: Sequential Dependency")
print("=" * 70)

# TODO: Uncomment and run after completing Part 2
# state = {"messages": [HumanMessage(content="""
# If I invest ₹50,000 at 7% compound interest for 3 years with quarterly 
# compounding, then take a loan for the final amount at 9% interest for 
# 2 years, what would be my monthly EMI?
# """)]}
# 
# for event in app.stream(state):
#     for node_name, data in event.items():
#         print(f"\n[Node: {node_name}]")
#         if "messages" in data:
#             last_msg = data["messages"][-1]
#             if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
#                 for tc in last_msg.tool_calls:
#                     print(f"  🔧 Calling: {tc['name']}")
# 
# result = app.invoke(state)
# print("\nFINAL ANSWER:")
# print(result["messages"][-1].content)

print("\n💡 Analyze the execution trace:")
print("   • Did compound_interest_calculator run first?")
print("   • Did emi_calculator use the compound interest result?")
print("   • How many agent → tools cycles occurred?")

---
# SECTION 8: Conclusion & Next Steps

## 🎉 Congratulations!

You've successfully built an agentic financial assistant using LangGraph!

### What You Accomplished

✅ **Created Custom Tools**
   - Currency converter with multi-currency support
   - EMI calculator with proper financial formula
   - Compound interest calculator (in exercise)

✅ **Built LangGraph Workflow**
   - Defined StateGraph with MessagesState
   - Implemented agent node (LLM decision-maker)
   - Created tool execution node
   - Configured conditional routing

✅ **Mastered Execution Patterns**
   - Single tool calls
   - Sequential multi-step reasoning
   - Dependent task chaining
   - Conversational context maintenance
   - Parallel tool execution

✅ **Understood Agentic Behavior**
   - LLM autonomously decides tool usage
   - Automatic sequential vs parallel detection
   - State-driven conversational flow
   - Cyclic execution for complex tasks

### Key Concepts Learned

In [ ]:
# Render Mermaid diagram
render_mermaid('''mindmap
  root((LangGraph))
    State Management
      MessagesState
      Message Types
      Immutable Updates
    Nodes
      Agent Node
      Tools Node
      Custom Functions
    Edges
      Static Edges
      Conditional Edges
      Routing Logic
    Execution
      Cycles
      Streaming
      Parallel Tools''')

### Architecture Pattern

```
User Query → State → Agent (LLM) → [Tools?] → Response
                         ↑              ↓
                         └──────────────┘
                         (Loop until satisfied)
```

This pattern applies to:
- Research agents (search → analyze → summarize)
- Customer support bots (lookup → execute → respond)
- Data analysis assistants (query → compute → visualize)
- Code assistants (analyze → execute → debug)

## Next Steps: Advancing Your LangGraph Skills

### 1. Observability with LangSmith

Access your execution traces:
- Visit LangSmith Dashboard
- View detailed token usage, latency, costs
- Debug complex execution flows
- Optimize performance bottlenecks

### 2. Advanced LangGraph Patterns

**Human-in-the-Loop**
- Pause execution for approval
- Allow manual tool selection
- Implement safety checks

**Sub-Graphs**
- Compose smaller graphs into larger workflows
- Modular architecture for complex systems

**Persistence**
- Checkpoint state to resume later
- Long-running workflows across sessions

**Error Handling**
- Retry logic for failed tools
- Fallback paths for degraded service
- Graceful error recovery

### 3. Extend This Application

**Additional Tools:**
- Stock price fetcher (real-time API)
- Tax calculator (based on income brackets)
- Investment portfolio analyzer
- Loan comparison tool

**Enhanced Agent:**
- Add memory beyond current session
- Implement user preferences
- Multi-turn planning for complex financial scenarios

**Production Readiness:**
- Input sanitization and validation
- Rate limiting for tool calls
- Caching for repeated queries
- Authentication and authorization

### 4. Real-World Project Ideas

1. **Research Assistant**
   - Web search → Summarization → Citation management
   
2. **Code Review Bot**
   - Static analysis → Test execution → Report generation

3. **Travel Planner**
   - Flight search → Hotel booking → Itinerary optimization

4. **Customer Support Agent**
   - Ticket lookup → Knowledge base search → Response generation

### 5. Learn More

**Documentation:**
- LangGraph Docs: https://langchain-ai.github.io/langgraph/
- LangChain Docs: https://python.langchain.com/

**Community:**
- LangChain Discord
- GitHub Discussions
- Stack Overflow `langgraph` tag

---

## 🚀 You're Ready to Build Agentic Applications!

The patterns you learned here apply to virtually any LLM-powered workflow. Start small, iterate, and gradually build complexity.

**Remember:**
- Tools extend LLM capabilities
- State enables conversational context
- Routing creates dynamic behavior
- Cycles allow iterative reasoning

Now go build something amazing! 🎯

## Common Issues & Solutions

### Issue 1: Tool Not Being Called

**Symptom:** Agent responds without using tools, even when needed

**Causes:**
- Tool docstring unclear or missing
- LLM doesn't understand when to use tool
- Tool parameters don't match query

**Solutions:**
1. Improve docstring with examples
2. Explicitly mention tool capabilities in system prompt
3. Use more descriptive parameter names
4. Test tool schema: `print(tool.args_schema.schema())`

---

### Issue 2: Graph Loops Infinitely

**Symptom:** Execution never reaches END, keeps cycling

**Causes:**
- Router always returns "tools"
- LLM never satisfied with results
- Conditional logic incorrect

**Solutions:**
1. Add cycle limit: `app = workflow.compile(recursion_limit=10)`
2. Check router logic in `should_continue`
3. Verify LLM receives tool results
4. Lower temperature for more consistent behavior

---

### Issue 3: Tool Execution Errors

**Symptom:** ToolMessage contains error text

**Causes:**
- Invalid parameters from LLM
- Missing input validation
- Type mismatch

**Solutions:**
1. Add robust error handling in tools
2. Validate inputs at tool entry
3. Return descriptive error messages
4. Check tool binding: `llm_with_tools.bind_tools(tools)`

---

### Debugging Tips

**1. Enable Verbose Mode**
```python
llm = ChatVertexAI(..., verbose=True)
```

**2. Print State at Each Step**
```python
for event in app.stream(state):
    print(f"State: {event}")
```

**3. Inspect Tool Calls**
```python
ai_message = result["messages"][-2]  # Second-to-last message
if hasattr(ai_message, "tool_calls"):
    print(ai_message.tool_calls)
```

**4. Check Message Types**
```python
for msg in result["messages"]:
    print(f"{type(msg).__name__}: {msg.content[:50]}")
```